In [63]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate
import functools


In [64]:
logging.basicConfig(level=logging.DEBUG)

## Nearest Neighbor (Greedy)

In [65]:
def nearest_neighbor_tsp(start_city_index=0):
    """
    Solves the TSP using the Nearest Neighbour Greedy algorithm.
    :param start_city_index: Index of the starting city
    :return: List of the order of the visited cities and total length of the route
    """
    dist_matrix=DIST_MATRIX.copy()
    visited = np.full(len(CITIES), False)  
    city = start_city_index  
    visited[city] = True

    tsp = [city]  

    while not np.all(visited):
        dist_matrix[:, city] = np.inf
        closest_city = np.argmin(dist_matrix[city])
        visited[closest_city] = True
        tsp.append(int(closest_city))
        city = closest_city

    tsp.append(start_city_index)
    
    total_distance = tsp_cost(tsp,DIST_MATRIX)
    
    return tsp, total_distance

In [66]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper


In [67]:
def valid(tsp):
    """
    It verifies the validity of a TSP route.
    """
    tsp = np.array(tsp)
    
    if tsp[0] != tsp[-1]:
        return False
    
    # Verify that all cities are visited exactly once.
    cities_visited = tsp[:-1]
    expected_cities = np.arange(len(CITIES))
    
    return (len(np.unique(cities_visited)) == len(CITIES) and 
            np.all(np.isin(expected_cities, cities_visited)))
@counter
def tsp_cost(route, dist_matrix):
    """Calculate the cost of the route and its validity."""
    route = np.array(route)
    # Calculates distances between consecutive cities.
    total_distance = np.sum([dist_matrix[route[i], route[i+1]] 
                           for i in range(len(route)-1)])
    return (total_distance, valid(route))

## Inversion mutation

In [68]:
def tweak(tsp):
    """Performs an inversion mutation on a TSP solution"""
    tsp = np.array(tsp)
    # Select two random points in the path to perform the inversion.
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    # Reverses the sub-path between the two selected points.
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


##  Simulated Annealing (SA) by Initial Population.

In [69]:
def simulated_annealing_population(dist_matrix, initial_temp, cooling_rate, stop_temp, population_size=100):
    """Simulated Annealing algorithm to generate an initial population"""
    dist_matrix = np.array(dist_matrix)
     # Find an initial solution with the Greedy method.
    current_solution, current_cost = nearest_neighbor_tsp(start_city_index=0)
    best_solution = current_solution[:]
    best_cost = current_cost

    population = [(best_solution, best_cost)]
    temperature = initial_temp

    while temperature > stop_temp:
        new_solution = tweak(current_solution[:])
        new_cost = tsp_cost(new_solution, dist_matrix)

        if (new_cost[0] < current_cost[0] or 
            np.random.random() < np.exp((current_cost[0] - new_cost[0]) / temperature)):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost[0] < best_cost[0]:
                best_solution = new_solution
                best_cost = new_cost
    
            population.append((current_solution, current_cost))
            
        temperature *= cooling_rate
    # Sort the population by cost and return the best 'population_size' individuals    
    population = np.array(population, dtype=object)
    sorted_indices = np.argsort([cost[0] for _, cost in population])
    population = population[sorted_indices]
    
    return population[:population_size].tolist()

In [70]:
def initial_population(pop_size):
    """Initialises the population using predefined parameters."""
    initial_temp = 1000
    cooling_rate = 0.992
    stop_temp = 1e-200
    # Generate initial population with Simulated Annealing.
    return simulated_annealing_population(DIST_MATRIX, initial_temp,
                                        cooling_rate, stop_temp, pop_size)


##  Crossover Inver-Over

In [71]:
def inver_over_crossover(parent1, parent2, num_iterations=1):
    """Apply inver-over crossover """
    parent1, parent2 = np.array(parent1), np.array(parent2)
    child = parent1.copy()
 # Perform a series of crossover iterations to create the child
    for _ in range(num_iterations):
        start_index = np.random.randint(1, len(child) - 2)
        start_city = child[start_index]
        # Randomly select a destination city from one of the parents.
        if np.random.random() < 0.5:
            valid_cities = child[1:-1][child[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        else:
            valid_cities = parent2[1:-1][parent2[1:-1] != start_city]
            end_city = np.random.choice(valid_cities)
        # Get start and destination city locations in the son.
        start_pos = np.where(child == start_city)[0][0]
        end_pos = np.where(child == end_city)[0][0]
        
        if start_pos > end_pos:
            start_pos, end_pos = end_pos, start_pos
          # Reverse the sub-path between the two positions.
        child[start_pos:end_pos + 1] = np.flip(child[start_pos:end_pos + 1])
    # The child ends up in the same city as the starting city.
    if child[0] != child[-1]:
        child = np.append(child, child[0])
    
    return child.tolist()

In [72]:
def tournament_selection(population, k=5):
    """Tournament selection using NumPy."""
    # Randomly selects 'k' individuals from the population.
    
    selected_indices = np.random.choice(len(population), k, replace=False)
    selected = np.array(population, dtype=object)[selected_indices]
    # Returns the individual with the lowest cost among those selected.
    return min(selected, key=lambda x: x[1][0])  # x[1][0] accede al costo


## Inversion mutation

In [73]:
def inversion_mutation(tsp):
    """Performs an inversion mutation """
    tsp = np.array(tsp)
    a, b = sorted(np.random.choice(range(1, len(tsp) - 1), 2, replace=False))
    tsp[a:b+1] = np.flip(tsp[a:b+1])
    return tsp.tolist()


## EA with adaptive mutation and multiple restart with simulated anneling.

In [74]:
def calculate_population_diversity(pop):
        """
        Calculates diversity by comparing unique solutions.
        Converts each path into a tuple to make it hashable.
        """
        # Extracts only paths (not costs) and converts them into tuples
        solutions = [tuple(individual[0]) for individual in pop]
        unique_solutions = set(solutions)
        
        # Calculate the proportion of unique solutions
        diversity = len(unique_solutions) / len(pop)
        return diversity

    
def generate_diverse_individual(dist_matrix):
        """Generate a completely new random solution"""
        cities_r = list(range(len(dist_matrix)))
        np.random.shuffle(cities_r)
        solution = cities_r + [cities_r[0]]  # Add the return to start
        return solution, tsp_cost(solution, dist_matrix)


In [ ]:
def evolutionary_algorithm_tsp_adaptive_mutation(
    dist_matrix,
    population_size=150,
    tournament_size=10,
    num_generations=10000,
    initial_mutation_rate=0.1,
    crossover_iterations=1,
    max_generations_without_improvement=500,
    mutation_increase_factor=1.5,
    mutation_decrease_factor=0.7,
    mutation_threshold=1.3,
    initial_temp=1000,
    cooling_rate=0.992,
    stop_temp=1e-200,
    diversity_threshold=0.3  
):
    """It solves the TSP using a Evolutionary Algorithm with adaptive mutation and diversity maintenance."""
    dist_matrix = np.array(dist_matrix)
    population = initial_population(population_size)
    current_mutation_rate = initial_mutation_rate
    
    best_solution, best_cost = min(population, key=lambda x: x[1][0])
    all_time_best_solution = best_solution
    all_time_best_cost = best_cost
    generations_without_improvement = 0
    

    
    for generation in tqdm(range(num_generations)):
        # Calculate current population diversity
        diversity = calculate_population_diversity(population)
        
        # If diversity is too low, inject new random solutions
        if diversity < diversity_threshold:
            num_new_solutions = population_size // 4  # Replace 25% of population
            new_solutions = [generate_diverse_individual(dist_matrix) for _ in range(num_new_solutions)]
            
            # Replace worst solutions with new random ones
            population = sorted(population, key=lambda x: x[1][0])
            population = population[:-num_new_solutions] + new_solutions
        
        
        # Check if it is necessary to reboot with SA
        if current_mutation_rate > mutation_threshold:
            print(f"\nMutation rate ({current_mutation_rate:.3f}) exceeded threshold ({mutation_threshold})")
            print("Restarting with Simulated Annealing...")
            
            new_population = simulated_annealing_population(
                dist_matrix,
                initial_temp,
                cooling_rate,
                stop_temp,
                population_size
            )
            num_new_solutions = int(population_size * 0.7)
            population = sorted(new_population, key=lambda x: x[1][0])[:population_size]
            population = population[:-num_new_solutions] + new_population
            current_mutation_rate = initial_mutation_rate
            print("Population refreshed and mutation rate reset")
        
        new_population = []
        
        # Modified selection to maintain diversity
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            
            child = inver_over_crossover(parent1[0], parent2[0], crossover_iterations)
            
            if np.random.random() < current_mutation_rate:
                child = inversion_mutation(child)

            child_cost = tsp_cost(child, dist_matrix)
            new_population.append((child, child_cost))

        combined_population = np.array(new_population + population, dtype=object)
        sorted_indices = np.argsort([cost[0] for _, cost in combined_population])
        population = combined_population[sorted_indices][:population_size].tolist()

        new_best_solution, new_best_cost = min(population, key=lambda x: x[1][0])

         # Update the best result ever if needed
        if new_best_cost[0] < all_time_best_cost[0]:
            all_time_best_solution = new_best_solution
            all_time_best_cost = new_best_cost

        if new_best_cost[0] < best_cost[0]:
            best_solution, best_cost = new_best_solution, new_best_cost
            generations_without_improvement = 0
            current_mutation_rate = max(initial_mutation_rate, 
                                     current_mutation_rate * mutation_decrease_factor)
        else:
            generations_without_improvement += 1

        if generations_without_improvement > max_generations_without_improvement:
            current_mutation_rate *= mutation_increase_factor
            generations_without_improvement = 0

        if generation % 1000 == 0:
            print(f"Generation {generation}: "
                  f"Absolute best cost = {all_time_best_cost[0]:.2f} | "
                  f"Mutation rate = {current_mutation_rate:.3f} | "
                  f"Diversity = {diversity:.3f} | "
                  f"Valid = {best_cost[1]}")
    return all_time_best_cost

## Vanuatu

In [76]:

CITIES = pd.read_csv('./cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=0, 
)
ic(best_cost,tsp_cost.calls)



0it [00:00, ?it/s]
ic| best_cost: (np.float64(1345.5449564733112), np.True_)
    tsp_cost.calls: 58196


((np.float64(1345.5449564733112), np.True_), 58196)

For vanuatu just applying simulated anneling to get the best solution is enough

## Italy

In [83]:

CITIES = pd.read_csv('./cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    mutation_increase_factor=1.3,
    mutation_decrease_factor=0.7
)
ic(best_cost,tsp_cost.calls)


  0%|          | 10/10000 [00:00<04:02, 41.22it/s]

Generation 0: Absolute best cost = 4436.03 | Mutation rate = 0.100 | Diversity = 0.987 | Valid = True


 10%|█         | 1005/10000 [00:24<03:49, 39.17it/s]

Generation 1000: Absolute best cost = 4258.07 | Mutation rate = 0.130 | Diversity = 0.273 | Valid = True


 20%|██        | 2007/10000 [00:49<03:20, 39.77it/s]

...

Generation 8000: Absolute best cost = 4172.76 | Mutation rate = 0.140 | Diversity = 0.267 | Valid = True


 90%|█████████ | 9008/10000 [03:54<00:26, 37.54it/s]

Generation 9000: Absolute best cost = 4172.76 | Mutation rate = 0.237 | Diversity = 0.240 | Valid = True


100%|██████████| 10000/10000 [04:20<00:00, 38.33it/s]
ic| best_cost: (np.float64(4172.7626139164095), np.True_)
    tsp_cost.calls: 17225581


((np.float64(4172.7626139164095), np.True_), 17225581)

## Russia

In [86]:

CITIES = pd.read_csv('./cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(DIST_MATRIX,mutation_threshold=1)
ic(best_cost,tsp_cost.calls)


  0%|          | 4/10000 [00:00<05:27, 30.49it/s]

Generation 0: Absolute best cost = 35920.20 | Mutation rate = 0.100 | Diversity = 1.000 | Valid = True


 10%|█         | 1007/10000 [00:37<04:58, 30.09it/s]

Generation 1000: Absolute best cost = 35279.71 | Mutation rate = 0.100 | Diversity = 0.960 | Valid = True


 20%|██        | 2004/10000 [01:23<04:03, 32.81it/s]

...

Generation 8000: Absolute best cost = 34500.09 | Mutation rate = 0.100 | Diversity = 0.673 | Valid = True


 90%|█████████ | 9003/10000 [05:31<00:32, 30.77it/s]

Generation 9000: Absolute best cost = 34500.09 | Mutation rate = 0.225 | Diversity = 0.487 | Valid = True


100%|██████████| 10000/10000 [06:05<00:00, 27.33it/s]
ic| best_cost: (np.float64(34500.0850090654), np.True_)
    tsp_cost.calls: 20516709


((np.float64(34500.0850090654), np.True_), 20516709)

## Us

In [79]:

CITIES = pd.read_csv('./cities/us.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=20000)
ic(best_cost,tsp_cost.calls)

  0%|          | 3/20000 [00:00<13:33, 24.60it/s]

Generation 0: Absolute best cost = 46171.34 | Mutation rate = 0.100 | Diversity = 1.000 | Valid = True


  5%|▌         | 1005/20000 [00:42<13:23, 23.64it/s]

Generation 1000: Absolute best cost = 42900.16 | Mutation rate = 0.100 | Diversity = 0.987 | Valid = True


 10%|█         | 2004/20000 [01:24<12:57, 23.16it/s]

...

Population refreshed and mutation rate reset


 95%|█████████▌| 19004/20000 [25:29<00:46, 21.58it/s]

Generation 19000: Absolute best cost = 40657.45 | Mutation rate = 0.100 | Diversity = 0.993 | Valid = True


100%|██████████| 20000/20000 [26:15<00:00, 12.69it/s]
ic| best_cost: (np.float64(40657.4481932187), np.True_)
    tsp_cost.calls: 6748142


((np.float64(40657.4481932187), np.True_), 6748142)

## China

In [80]:

CITIES = pd.read_csv('./cities/china.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
best_cost=evolutionary_algorithm_tsp_adaptive_mutation(
    DIST_MATRIX,
    num_generations=30000, 
)
ic(best_cost,tsp_cost.calls)


  0%|          | 2/30000 [00:00<34:55, 14.32it/s]

Generation 0: Absolute best cost = 63962.92 | Mutation rate = 0.100 | Diversity = 1.000 | Valid = True


  3%|▎         | 1002/30000 [02:55<37:20, 12.94it/s] 

Generation 1000: Absolute best cost = 62312.90 | Mutation rate = 0.100 | Diversity = 1.000 | Valid = True


  7%|▋         | 2001/30000 [04:14<1:35:18,  4.90it/s]

...

Generation 28000: Absolute best cost = 52923.22 | Mutation rate = 0.100 | Diversity = 0.993 | Valid = True


 97%|█████████▋| 29003/30000 [52:12<01:08, 14.55it/s]

Generation 29000: Absolute best cost = 52923.22 | Mutation rate = 0.225 | Diversity = 0.993 | Valid = True


100%|██████████| 30000/30000 [53:21<00:00,  9.37it/s]
ic| best_cost: (np.float64(52913.04630995), np.True_)
    tsp_cost.calls: 11306338


((np.float64(52913.04630995), np.True_), 11306338)

The proposed evolutionary algorithm addresses the traveling salesman problem (TSP) by integrating an adaptive mutation strategy and a restart mechanism based on Simulated Annealing (SA). Mutation is dynamically regulated: the rate increases if no improvement is observed for a given number of generations and decreases if progress is made, thus promoting a balance between exploration and exploitation.
Unlike the first version I published now, I also take into account the diversity of the population by inserting random solutions if the population has too many equal solutions.
However, I did not achieve any major changes compared to the first version, obtaining similar results

When the mutation rate exceeds a predefined threshold, the algorithm initiates a restart phase via SA, regenerating the population to avoid convergence to local minima. The approach uses tournament selection, permutation-specific crossover techniques of the TSP (inver-over), and an inversion-based mutation. This combination allows the algorithm to progressively improve the quality of the solution.

Note: in order to obtain the same results, it may be necessary to restart the algorithm several times or to increase the gender number. The results shown on the screen are the highest values I obtained after several attempts